<a href="https://colab.research.google.com/github/murpunk/Programming_2023/blob/main/%D0%9F%D0%B0%D0%BD%D0%BA%D0%BE%D0%B2%D0%B0_4_NER_Natasha_Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание

1. Выбрать 2 небольших текста для анализа (до 10000 знаков) на английском и русском языках.
2. Сделать их перевод с помощью любимого машинного переводчика
3. Для каждого текста на русском языке извлечь именованные сущности (Natasha) и построить их частотный словарь. Упрощенно сущности считаются одинаковыми, если у них совпадают first, last и middle
4. Сделать то же самое для текстов на английском (Spacy). Там можно брать
entity.text в качестве текста сущности
5. Сравните результаты. Есть ли отличия? Если да, что могло повляить на их появление?

### 1. Установка библиотек и текстов

In [ ]:
!pip install natasha

In [ ]:
!pip install spacy

Загружаю тексты

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

В качестве одной из пары текстов — rus_orig.txt и rus_transl.txt — были выбраны рассказ Марии Коленовой "Дом", финалистки конкурса "Класс", одним из условий этого конкурса были тексты объёмом не более 10 000 символов, и перевод этого рассказа, выполненный в Яндекс.Переводчиком.
В качестве другой пары текстов — eng_orig.txt и eng_transl.txt — были выбраны короткий рассказ про вампира и молодую пару и перевод этого рассказа на русский язык, выполненный в Яндекс.Переводчиком.

In [ ]:
folder_path = "drive/My Drive/NER/"

files = os.listdir(folder_path)

stories_list = []

for file_name in files:
    file_path = os.path.join(folder_path, file_name)

    if os.path.isfile(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
            len_text = len(text)
            stories_list.append(text)

            print(f"{file_name}, количество символов: {len_text}")

rus_orig.txt, количество символов: 5287
rus_transl.txt, количество символов: 6067
eng_orig.txt, количество символов: 7265
eng_transl.txt, количество символов: 7427


In [ ]:
stories_list[0]

### 2. Работа с Natasha

In [ ]:
from natasha import (
    Segmenter, MorphVocab,
    NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger,
    NamesExtractor, DatesExtractor, MoneyExtractor, AddrExtractor,
    PER, Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [ ]:
def extract_names(text):
    """Извлекает имена из текста"""
    doc = Doc(text)
    doc.segment(segmenter)  # token, sentence segmentation
    doc.tag_morph(morph_tagger)   # lemmatization
    # doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)  # Ner extractor
    for span in doc.spans:
        if span.type == PER:
            span.normalize(morph_vocab)  # приводим к начальной форме
            span.extract_fact(names_extractor)  # extract names
    names = []
    for span in doc.spans:
        if span.fact:
            pers_info = {'normal': span.normal,
                         # 'start': span.start,
                         # 'end': span.stop,
                         'tokens': [{'start': _.start, 'end': _.stop} for _ in span.tokens],
                         'text': span.text
                         }
            pers_info.update(span.fact.as_dict)
            names.append(pers_info)
    return names

In [ ]:
def extract_address(text):
    """Извлекает адреса из текста"""
    doc = Doc(text)
    doc.segment(segmenter)  # token, sentence segmentation
    doc.tag_morph(morph_tagger)   # lemmatization
    # doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)  # Ner extractor
    for span in doc.spans:
#        if span.type == AddrExtractor:
         span.normalize(morph_vocab)  # приводим к начальной форме
         span.extract_fact(addr_extractor)  # извлекает адреса
    address = []
    for span in doc.spans:
        if span.fact:
            pers_info = {'normal': span.normal,
                         # 'start': span.start,
                         # 'end': span.stop,
                         'tokens': [{'start': _.start, 'end': _.stop} for _ in span.tokens],
                         'text': span.text
                         }
            pers_info.update(span.fact.as_dict)
            address.append(pers_info)
    return address

In [ ]:
list(names_extractor(stories_list[0]))

In [ ]:
extract_names(stories_list[0]) # список имён собственных в русском оригинальном тексте

In [ ]:
extract_names(stories_list[3]) # список имён собственных в переводе на русский язык

In [ ]:
# list(dates_extractor(text))

In [ ]:
# list(money_extractor(text))

In [ ]:
list(addr_extractor(stories_list[0])) # список географических наименований в русском оригинальном тексте

In [ ]:
list(addr_extractor(stories_list[3])) # список географических наименований в переводе на русский

[]

Составление словаря частотности имён в русских текстах: оригинальном и в переводе.

In [ ]:
rus_orig_dict = {}
for name in extract_names(stories_list[0]):
    if name["normal"] in rus_orig_dict:
        rus_orig_dict[name["normal"]] += 1
    else:
        rus_orig_dict[name["normal"]] = 1
rus_orig_dict

{'Питер': 1}

In [ ]:
eng_transl_dict = {}
for name in extract_names(stories_list[3]):
    if name["normal"] in eng_transl_dict:
        eng_transl_dict[name["normal"]] += 1
    else:
        eng_transl_dict[name["normal"]] = 1
eng_transl_dict

{'Дэвид': 26, 'Эмма': 17, 'Эмму': 1}

Составление словаря частотности географических локаций в русских текстах: оригинальном и в переводе.

In [ ]:
rus_orig_addr_dict = {}
for address in extract_address(stories_list[0]):
    if address["normal"] in rus_orig_addr_dict:
        rus_orig_addr_dict[address["normal"]] += 1
    else:
        rus_orig_addr_dict[address["normal"]] = 1
rus_orig_addr_dict

{'Москва': 2, 'Череповец': 2, 'Магнитогорск': 2}

In [ ]:
eng_trasl_addr_dict = {}
for address in extract_address(stories_list[3]):
    if name["normal"] in eng_trasl_addr_dict:
        eng_trasl_addr_dict[address["normal"]] += 1
    else:
        eng_trasl_addr_dict[address["normal"]] = 1
eng_trasl_addr_dict

{}

Объединяю словари имён и адресов.

In [ ]:
rus_orig_dict.update(rus_orig_addr_dict)
print(rus_orig_dict)

{'Питер': 1, 'Москва': 2, 'Череповец': 2, 'Магнитогорск': 2}


In [ ]:
eng_transl_dict.update(eng_trasl_addr_dict)
print(eng_transl_dict)

{'Дэвид': 26, 'Эмма': 17, 'Эмму': 1}


### 3. Работа со spacy

In [ ]:
!python -m spacy download en_core_web_md # загрузка модели

In [ ]:
import spacy           # подключение модели
import en_core_web_md

In [ ]:
nlp = en_core_web_md.load()

In [ ]:
rus_transl = nlp(stories_list[1])
for entity in rus_transl.ents:
        print("Entity: ", entity.text)
        print("Entity Type:" + entity.label_ + '|' + spacy.explain(entity.label_))
        print("--")

In [ ]:
eng_orig = nlp(stories_list[2])
for entity in eng_orig.ents:
        print("Entity: ", entity.text)
        print("Entity Type:" + entity.label_ + '|' + spacy.explain(entity.label_))
        print("--")

Составляю частотный словарь для перевода на английский и для оригинального текста на английском.

In [ ]:
rus_transl_dict = {}
for name in rus_transl.ents:
    if name.text in rus_transl_dict:
        rus_transl_dict[name.text] += 1
    else:
      rus_transl_dict[name.text] = 1

rus_transl_dict

{'Moscow': 2,
 'Cherepovets': 2,
 'an hour': 1,
 'Magnitogorsk': 2,
 'the Magnetic Mountain': 1,
 'Southern Urals': 1,
 'Two days': 1,
 'two thousand kilometers': 1,
 'ten years old': 1,
 'First': 1,
 'a hundred': 1,
 'Ninety-nine': 1,
 'ninety-seven': 1,
 'Europe': 1,
 'Asia': 1,
 'the Ural Ridge': 1,
 'midday': 1,
 'North': 2,
 'Urals': 1,
 'jasper': 1,
 'GEC': 1,
 'TPP': 1,
 'NPP': 1,
 'Bashkiria': 1,
 'fifty-five': 1,
 'fifty-four': 1,
 'fifty-three': 1,
 'Grandma': 2,
 'Edelweiss': 1,
 'the Copper Mountain': 1,
 'Twenty, nineteen,': 1,
 'eighteen': 1,
 'half': 1,
 'One': 5,
 'Winter day': 1,
 'winter': 1,
 'evening': 1,
 'night': 3,
 'St. Petersburg': 1,
 'this morning': 1,
 'Peter': 1,
 'every six months': 1,
 'two': 2,
 'three': 2,
 'Two': 2,
 'Three.': 1}

In [ ]:
eng_orig_dict = {}
for name in eng_orig.ents:
    if name.text in eng_orig_dict:
        eng_orig_dict[name.text] += 1
    else:
      eng_orig_dict[name.text] = 1

eng_orig_dict

{'David': 25,
 'Emma': 18,
 'One minute': 1,
 'two': 1,
 'third': 2,
 'tonight': 2,
 'only one': 1,
 'three': 1,
 'One': 1,
 'many centuries': 1}

### 4. Вывод результатов

Можно в одной таблице представить словари частотностей для русского и английского текстов.

In [ ]:
import pandas as pd


In [ ]:
# df = pd.DataFrame.from_dict(rus_orig_dict, orient='index', columns = ["Rus_orig", "count"])
# df

In [ ]:
df1 = pd.DataFrame.from_dict(list(rus_orig_dict))
# , columns=["Rus_orig","amount"]
df2 = pd.DataFrame.from_dict(list(rus_transl_dict))
# , columns=["Eng_transl_orig","amount"]

combined_df_1 = pd.concat([df1, df2], axis=1)
print(combined_df_1)

               0                        0
0          Питер                   Moscow
1         Москва              Cherepovets
2      Череповец                  an hour
3   Магнитогорск             Magnitogorsk
4            NaN    the Magnetic Mountain
5            NaN           Southern Urals
6            NaN                 Two days
7            NaN  two thousand kilometers
8            NaN            ten years old
9            NaN                    First
10           NaN                a hundred
11           NaN              Ninety-nine
12           NaN             ninety-seven
13           NaN                   Europe
14           NaN                     Asia
15           NaN           the Ural Ridge
16           NaN                   midday
17           NaN                    North
18           NaN                    Urals
19           NaN                   jasper
20           NaN                      GEC
21           NaN                      TPP
22           NaN                  

In [ ]:
rus_orig_dict

{'Питер': 1, 'Москва': 2, 'Череповец': 2, 'Магнитогорск': 2}

In [ ]:
rus_transl_dict

**Итог**: в русском тексте есть ошибка выделения именованной сущности, связанная с тем, что название города Питер посчиталось за имя человека. В английском тексте помимо других именованных сущностей, которые мы не рассматриваем (даты, числа, расстояния), библиотекой spacy выделено гораздо больше географических локаций: *Санкт-Петербург (St. Petersburg), Медная гора (the Copper Mountain), Башкирия (Bashkiria), Уральский хребет (the Ural Ridge), Южный Урал (Southern Urals), Магнитная гора (the Magnetic Mountain), Европа (Europe), Азия (Asia)*.

Слова *Питер (Peter)* и *Петербург (St. Petersburg)* не объединились в одну сущность.

При этом аббревиатуры, которые в русском тексте выглядят как *ГЭЦ, ТЭС, АЭС*, библиотека spacy также посчитала за именованные сущности. Туда же вошли слова *яшма (jasper), эдельвейс (Edelweiss)*, а также слово *Север (North)*, которое в русском тексте было записано с заглавной буквы.

По частотности выделенные русские и английские сущности совпадают.

In [ ]:
df3 = pd.DataFrame.from_dict(list(eng_orig_dict))
df4 = pd.DataFrame.from_dict(list(eng_transl_dict))

combined_df_2 = pd.concat([df3, df4], axis=1)
print(combined_df_2)

                0      0
0           David  Дэвид
1            Emma   Эмма
2      One minute   Эмму
3             two    NaN
4           third    NaN
5         tonight    NaN
6        only one    NaN
7           three    NaN
8             One    NaN
9  many centuries    NaN


In [ ]:
eng_orig_dict

{'David': 25,
 'Emma': 18,
 'One minute': 1,
 'two': 1,
 'third': 2,
 'tonight': 2,
 'only one': 1,
 'three': 1,
 'One': 1,
 'many centuries': 1}

In [ ]:
eng_transl_dict

{'Дэвид': 26, 'Эмма': 17, 'Эмму': 1}

**Итог**: в русском переводе не объединились в одну сущность формы имени у девушки: *Эмма* и *Эмму*. Также в русском переводе на один раз больше используется имя мужчины *Дэвид*. По всей видимости, это особенность перевода. Но так как больше сущностей в этих текстах не было (не считаем числа и обозначение времени), то более подробных выводов сделать сложно.